In [1]:
import pandas as pd
import sys
import logging
import os
import datetime as dt
sys.path.append('../src')
print(sys.path)  # Debug: check if '../src' is in sys.path
from utils.fetch_data import fetch_nwis_data
import utils.duckdb_utils as du
import utils.site_list as sl



['C:\\Users\\miesho\\AppData\\Roaming\\uv\\python\\cpython-3.12.9-windows-x86_64-none\\python312.zip', 'C:\\Users\\miesho\\AppData\\Roaming\\uv\\python\\cpython-3.12.9-windows-x86_64-none\\DLLs', 'C:\\Users\\miesho\\AppData\\Roaming\\uv\\python\\cpython-3.12.9-windows-x86_64-none\\Lib', 'C:\\Users\\miesho\\AppData\\Roaming\\uv\\python\\cpython-3.12.9-windows-x86_64-none', 'c:\\Users\\miesho\\Projects_git\\ucpo_waterdata\\.venv', '', 'c:\\Users\\miesho\\Projects_git\\ucpo_waterdata\\.venv\\Lib\\site-packages', 'c:\\Users\\miesho\\Projects_git\\ucpo_waterdata\\.venv\\Lib\\site-packages\\win32', 'c:\\Users\\miesho\\Projects_git\\ucpo_waterdata\\.venv\\Lib\\site-packages\\win32\\lib', 'c:\\Users\\miesho\\Projects_git\\ucpo_waterdata\\.venv\\Lib\\site-packages\\Pythonwin', '../src']


In [2]:
# Configure logging ------------------------------------------------
os.makedirs('logs', exist_ok=True)
log_name = 'logs/' + dt.datetime.now().strftime('%Y-%m-%d_%H-%M-%S') + '.log'
logging.basicConfig(filename=log_name,
                    level=logging.INFO,
                    format='%(asctime)s - %(levelname)s - %(message)s')
# -------------------------------------------------------------------

In [3]:
print(sl.nwis_sites)
sites = ", ".join(sl.nwis_sites)

['09152500', '09095500', '09106150', '09106485', '09163500', '09306500', '09251000', '09260050', '09260000', '09261000', '09315000', '09302000', '09180000', '09328960', '09147022', '09041395', '09379900']


In [4]:
nwis_rename_map = {
    'site_no': 'site_cd',
    'station_nm': 'site_nm',
    'agency_cd': 'agency_cd',
    'dec_lat_va': 'lat_dd',
    'dec_long_va': 'lon_dd',
    'alt_va': 'elev_ft',
    'site_tp_cd': 'site_type'
    }

bor_rename_map = {
    'site_id': 'site_cd', 'site_metadata.site_name': 'site_nm', 
    'site_metadata.lat': 'lat_dd', 'site_metadata.longi': 'lon_dd', 
    'site_metadata.elevation': 'elev_ft',

}

In [ ]:
nwis_metadata = fetch_nwis_data(
    site=sites,
    service_code='site')

selected = nwis_metadata[list(nwis_rename_map)].rename(columns=nwis_rename_map).reset_index(drop=True)
merged = selected.merge(sl.hydrologic_areas, on='site_cd', how='left')
merged['elev_m'] = (merged['elev_ft'] * 0.3048).round()
merged['lat_dd'] = merged['lat_dd'].round(7)
merged['lon_dd'] = merged['lon_dd'].round(7)
merged['agency_nm'] = 'US Geological Survey'
merged['site_type'] = merged['site_type'].str.replace('ST', 'Stream')
merged['site_type'] = merged['site_type'].str.replace('LK', 'Lake')
merged['source'] = 'NWIS API'
merged['site_dsc'] = None
merged['site_id'] = range(1, len(merged) + 1)
merged['create_ts'] = dt.datetime.now()
merged['update_ts'] = dt.datetime.now()

nwis_metadata_cleaned = merged[
    ['site_id', 'site_cd', 'site_nm', 'site_dsc', 'agency_cd', 'agency_nm', 
    'lat_dd', 'lon_dd', 'elev_m', 'site_type', 'hydro_area_cd',
    'hydro_area_nm', 'source', 'create_ts', 'update_ts']].sort_values(by='site_cd')


In [5]:
bor_metadata = pd.read_csv("https://www.usbr.gov/uc/water/hydrodata/reservoir_data/meta.csv")
bor_selected = bor_metadata[list(bor_rename_map)].rename(columns=bor_rename_map).reset_index(drop=True)
bor_selected['site_cd'] = bor_selected['site_cd'].astype(str)
bor_filtered = bor_selected[bor_selected['site_cd'].isin(sl.bor_sites)].drop_duplicates(subset='site_cd')

bor_merged = bor_filtered.merge(sl.hydrologic_areas, on='site_cd', how='left')
bor_merged['elev_m'] = (bor_merged['elev_ft'] * 0.3048).round()
bor_merged['lat_dd'] = bor_merged['lat_dd'].round(7)
bor_merged['lon_dd'] = bor_merged['lon_dd'].round(7)
bor_merged['agency_nm'] = 'US Bureau of Reclamation'
bor_merged['agency_cd'] = 'BOR'
bor_merged['site_type'] = "Lake"
bor_merged['source'] = 'https://www.usbr.gov/uc/water/hydrodata/reservoir_data/<site_cd>/csv/<parameter_cd>.csv'
bor_merged['site_dsc'] = None
bor_merged['site_id'] = range(18, len(bor_merged) + 18)
bor_merged['create_ts'] = dt.datetime.now()
bor_merged['update_ts'] = dt.datetime.now()

bor_metadata_cleaned = bor_merged[
    ['site_id', 'site_cd', 'site_nm', 'site_dsc', 'agency_cd', 'agency_nm', 
    'lat_dd', 'lon_dd', 'elev_m', 'site_type', 'hydro_area_cd',
    'hydro_area_nm', 'source', 'create_ts', 'update_ts']]



In [3]:
with du.connect_duckdb() as con:
    # Verify the data was inserted
    result = con.execute("SELECT MAX(site_id) FROM site").fetchone()
    print(f"max site_id from 'site' table: {result[0]}")
    #logging.info(f"Number of rows in 'site' table: {result[0]}")


max site_id from 'site' table: 27


In [4]:
cbrfc_metadata = sl.cbrfc_sites_df.copy()
cbrfc_metadata['site_id'] = range(28, len(cbrfc_metadata) + 28)
cbrfc_metadata['create_ts'] = dt.datetime.now()
cbrfc_metadata['update_ts'] = dt.datetime.now()
cbrfc_metadata_cleaned = cbrfc_metadata[
    ['site_id', 'site_cd', 'site_nm', 'site_dsc', 'agency_cd', 'agency_nm', 
    'lat_dd', 'lon_dd', 'elev_m', 'site_type', 'hydro_area_cd',
    'hydro_area_nm', 'source', 'create_ts', 'update_ts']]

In [ ]:
print(sl.hydrologic_areas)
ck = pd.read_csv(f"https://www.usbr.gov/uc/water/hydrodata/reservoir_data/{bor_merged['site_cd'][0]}/csv/17.csv")

In [ ]:
du.run_sql_file('../db/schema.sql')

In [5]:
query = """
INSERT INTO site
SELECT * FROM staging_table
"""
print(query)


INSERT INTO site
SELECT * FROM staging_table



In [ ]:
with du.connect_duckdb() as con:
    columns = con.execute("PRAGMA table_info('site')").fetchall()
    print(columns)

In [6]:
with du.connect_duckdb() as con:
        try:
            con.register('staging_table', cbrfc_metadata_cleaned)
            con.execute(query)
            logging.info(f"✅ Successfully executed SQL query: {query}")
        except Exception as e:
            logging.error(f"❌ Error executing SQL file {query}: {e}")
            raise


In [16]:
with du.connect_duckdb() as con:
    # Verify the data was inserted
    result = con.execute("SELECT COUNT(*) FROM site").fetchone()
    print(f"Number of rows in 'site' table: {result[0]}")
    logging.info(f"Number of rows in 'site' table: {result[0]}")


Number of rows in 'site' table: 45


In [7]:
with du.connect_duckdb() as con:
    # Verify the data was inserted
    result = con.execute("SELECT * FROM site").df()

In [21]:
result.to_csv('site_metadata.csv', index=False)